<a href="https://colab.research.google.com/github/sahug/ds-tensorflow-colab/blob/master/Tensorflow%20-%20Multi-Label%20Image%20Classification%20Using%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tensorflow - Multi-Label Image Classification Using CNN**

In [ ]:
!pip install tensorflow-gpu==2.0.0

Movie poster link - https://www.cs.ccu.edu.tw/~wtchu/projects/MoviePoster/

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Conv2D, MaxPool2D

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image

In [ ]:
print(tf.__version__)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
# Clonning data from repository
!git clone https://github.com/laxmimerit/Movies-Poster_Dataset.git

In [ ]:
data = pd.read_csv("/content/Movies-Poster_Dataset/train.csv")
data.shape

In [ ]:
data.head() #ID Correspondes to Image Name in the Images Directory

In [ ]:
# For Processing we need the data/image in same size. Converting all images to same size
img_width = 200
img_height = 200

x = []

for i in tqdm(range(data.shape[0])):
  path = "/content/Movies-Poster_Dataset/Images/" + data["Id"][i] + ".jpg"
  img = image.load_img(path, target_size=(img_width, img_height, 3)) #Reshapping image. 3 is for color.
  img = image.img_to_array(img) #Converting image to array
  img = img/255.0 #Bringing down variance to common difference
  x.append(img)  

x = np.array(x)

In [ ]:
x.shape

In [ ]:
plt.imshow(x[1])

In [ ]:
data["Genre"][1]

In [ ]:
# Prepare Train and Test Data
y = data.drop(["Id", "Genre"], axis = 1)
y = y.to_numpy()
y.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=0)

**Build CNN**

In [ ]:
model = Sequential()

# Layer 1
model.add(Conv2D(16, (3, 3), activation="relu", input_shape=x_train[0].shape))
model.add(BatchNormalization())
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.3))

# Layer 2
model.add(Conv2D(32, (3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.3))

# Layer 3
model.add(Conv2D(64, (3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.4))

# Layer 4
model.add(Conv2D(128, (3, 3), activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(2, 2))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(128, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(128, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(25, activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [ ]:
history.history

In [ ]:
epoch_range = range(1, 6)

In [ ]:
# Plot training and validation accuracy values
# sparse_categorical_accuracy vs val_sparse_categorical_accuracy
plt.plot(epoch_range, history.history["accuracy"])
plt.plot(epoch_range, history.history["val_accuracy"])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(["Train", "Val"], loc="upper left")
plt.show()

In [ ]:
# Plot training and validation loss values
# Loss vs Val_Loss
plt.plot(epoch_range, history.history["loss"])
plt.plot(epoch_range, history.history["val_loss"])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(["Train", "Val"], loc="upper left")
plt.show()

**Making Predictions**

In [ ]:
img = image.load_img("/content/Movies-Poster_Dataset/endgame.jpg", target_size=(img_width, img_height, 3))
plt.imshow(img)
img = image.img_to_array(img)
img = img/255.0
img = img.reshape(1, img_width, img_height, 3)


In [ ]:
classes = data.columns[2:]
classes

In [ ]:
y_prob = model.predict(img)
y_prob, y_prob[0]

In [ ]:
#Sorting and getting top 3 predictions
top_3_predictions = np.argsort(y_prob[0])[:-4:-1]
top_3_predictions

In [ ]:
for i in range(3):
  print(classes[top_3_predictions[i]])